In [11]:
import sys
import logging
import os.path as p
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
import print_n_log

from sklearn.metrics import classification_report, confusion_matrix
from torch.utils.data.dataloader import DataLoader
from definitions import *
from model_helper_functions import *
from dataset_helper_functions import *
from bi_lstm import BiLSTM
from bert_embedding_model import BertEmbeddingModel
from debates_dataset import DebatesDataset
from early_stopping import EarlyStopping
from optuna.trial import TrialState
from torchvision import transforms
# my transforms
from transforms import *

In [13]:
data = {}
optim_path = os.path.join(EXP_DIR_PATH, 'bi-lstm', 'optimization')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_uw_ratio = 0
dataset_frac = 0.2
worthy_frac = 0.2
slf_dim = 0
# stopwords_type = 'wstop'
rs = 22

Function for loading data.

In [17]:
def load_data():
    global data
    dev_path = p.join(PROC_DATA_DIR_PATH, 'dev')

    data_paths = {
        'dev': [
            p.join(dev_path, 'dev.tsv'),
        ],
        'test': [
            p.join(POLIT_DATA_DIR_PATH, 'test', 'test_combined.tsv'),
        ],
        'train': [
            p.join(POLIT_DATA_DIR_PATH, 'train', 'train_combined.tsv'),
        ],
        'val': [
            p.join(POLIT_DATA_DIR_PATH, 'val', 'val_combined.tsv'),
        ],
    }

    for dtype, dpaths in data_paths.items():
        try:
            data[dtype] = pd.read_csv(dpaths[0], sep='\t', index_col=False)

        except Exception as e:
            print(e.args)
            exit()



11665
train p ratio:  0.028975567938276898
val p ratio:  0.026227822062226794
Empty DataFrame
Columns: [i, id, src, content, label]
Index: []


In [5]:
# stopwords_type = 'wostop'
# from_sel = True
# transform_pipeline = transforms.Compose([
#     HandleStopwords(stopwords=stopwords_type),
#     NoTransform('pos', from_selection=from_sel, stopwords=stopwords_type),
#     OneHot('tag', from_selection=from_sel, stopwords=stopwords_type),
#     NoTransform(),
#     ToBinary(6),
#     ToTensor()
# ])
# dd = DebatesDataset(data=filter_by_length(data['train']), transform=transform_pipeline)
# # dd = pad_features(dd)
# # for ids, content, label, feature in dd:
# #     print(content)
# #     print(feature.size())
# dl = DataLoader(dd, batch_size=32, shuffle=True, drop_last=True)

# for ids, sentences, labels, features in dl:
# #     print(features.size())
#     pass
# print('done')

Datasets and DataLoaders, takes trial as input to be able to suggest values for variables.

In [6]:
def get_loaders(trial, stopwords_type):
    global train_uw_ratio, slf_dim
    # dev_df, test_df, train_df, val_df = data.values()
    subsets = {}
    for k, df in data.items():

        n_subset = int(len(df)*dataset_frac)

        worthy_df = df.loc[df['label'] == 1]
        n_worthy = min(int(n_subset*worthy_frac), len(worthy_df))
        worthy_df = worthy_df.sample(n=n_worthy, random_state=rs)

        unworthy_df = df.loc[df['label'] == 0].sample(
            n=n_subset-n_worthy,
            random_state=rs
        )
        if k == 'train':
            train_uw_ratio = len(unworthy_df) / len(worthy_df)
        # sample(frac=1.0) -> shuffle
        subsets[k] = worthy_df.append(unworthy_df).sample(frac=1.0, random_state=rs, ignore_index=True)
    

    transforms_map = {
        'sum': Sum,
        'onehot': OneHot,
        'none': NoTransform
    }
    transforms_options = list(transforms_map.keys())
    cw_map = {
        'count_words': CountWords,
        'none': NoTransform
    }
    cw_options = list(cw_map.keys())

    from_sel = trial.suggest_categorical('from_selection', [True, False])

    # trial.suggest_categorical returns one of the keys of transforms_map, which then return Transform or None
    # if transform --> initialize
    pos_feat = transforms_map[trial.suggest_categorical('pos_feature_type', transforms_options)]
    pos_feat = pos_feat(
        'pos', from_selection=from_sel, stopwords=stopwords_type
    )

    tag_feat = transforms_map[trial.suggest_categorical('tag_feature_type', transforms_options)]
    tag_feat = tag_feat(
        'tag', from_selection=from_sel, stopwords=stopwords_type
    )

    cw_feat = cw_map[trial.suggest_categorical('word_count_feature_type', cw_options)]
    cw_feat = cw_feat()
    
    transform_pipeline = transforms.Compose([
        HandleStopwords(stopwords=stopwords_type),
        pos_feat,
        tag_feat,
        cw_feat,
        ToBinary(6),
        ToTensor()
    ])
    # transform_pipeline = None

    train_dd = DebatesDataset(data=subsets['train'], transform=transform_pipeline)
    val_dd = DebatesDataset(data=subsets['val'], transform=transform_pipeline)
    test_dd = DebatesDataset(data=subsets['test'], transform=transform_pipeline)

    slf_dim = train_dd[0][-1].size()[0]
    # batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
    batch_size = 32

    train_loader = DataLoader(train_dd, batch_size=batch_size, shuffle=True, drop_last=True)
    val_loader = DataLoader(val_dd, batch_size=batch_size, shuffle=True, drop_last=True)
    test_loader = DataLoader(test_dd, batch_size=batch_size, shuffle=True, drop_last=True)

    return train_loader, val_loader, test_loader

Model setup + training loop

In [7]:
def objective(trial):
    global logf_path
    # this is here so that it can be accessed here and in get_loaders()
    stopwords_type = trial.suggest_categorical('stopwords_type', ['wstop', 'wostop'])

    train_loader, val_loader, test_loader = get_loaders(trial, stopwords_type)

    # hyperparams opt
    # pooling_strategy = trial.suggest_categorical('pooling_strategy', ['last_four', 'last_four_sum', 'second_last'])
    # dropout = trial.suggest_float('dropout', 0.0, 0.5, step=0.01)
    # hidden_dim = trial.suggest_categorical('hidden_dim', [128, 256, 512])
    # w_seq = trial.suggest_categorical('with_sequential_layer', [True, False])
    # lr = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    # opt_weight_decay = trial.suggest_float('optimizer_weigth_decay', 1e-6, 0.1, log=True)
    # pos_weight = trial.suggest_categorical('pos_weight', [1.0, train_uw_ratio])

    # temp_best
    pooling_strategy = 'second_last'
    dropout = 0.39
    hidden_dim = 256
    w_seq = True
    lr = 0.004118121
    opt_weight_decay = 0.024460049
    pos_weight = train_uw_ratio
    

    # should_scale_emb = trial.suggest_categorical('should_scale_emb', [True, False])
#     pooling_strategy = 'second_last'

    # TODO: test these as well
    # remove_stopwords = stopwords_type != 'wstop'
    # dep_feat = trial.suggest_categorical('word_level_dep_features', [True, False])
    # triplet_feat = trial.suggest_categorical('word_level_triplet_features', [True, False])
#     word_level_feat = trial.suggest_categorical('word_level_feature_type', ['dep', 'triplet'])
    embedding_model = BertEmbeddingModel(
        device=device,
        pooling_strat=pooling_strategy,
        scale=False,
        dep_features=False,#word_level_feat == 'dep',
        triplet_features=False,#word_level_feat == 'triplet',
        remove_stopwords=False,#stopwords_type == 'wostop'
    )
    
#     lstm dropout only works with multiple lstm layers
#     lstm_dropout = trial.suggest_float('lstm_dropout', 0.0, 0.3, step=0.05)
#     dropout = 0.4
#     lstm_dropout = 0.0
#     hidden_dim = 128
    model = BiLSTM(
        dropout=dropout,
        hidden_dim=hidden_dim,
        embedding_dim=embedding_model.dim,
        sent_level_feature_dim=slf_dim,
        device=device,
        w_seq=w_seq,
    ).to(device)    

#     lr = 0.001
#     opt_weight_decay = 0
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=opt_weight_decay)

#     pos_weight = train_uw_ratio
    criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos_weight]).to(device))

    n_epochs = 16
    threshold = 0.5
    early_stopping = EarlyStopping(
        patience=5,
        path=None,
        verbose=False,
        trace_func=print_n_log.run('early_stopping', logf_path, 'DEBUG')
    )
    
    # training
    for epoch in range(n_epochs):
        print(epoch)
        losses, val_losses = [], []

        model.train()
        for ids, sentences, labels, features in train_loader:
            labels = labels.float().to(device)
            features = features.to(device)
            
            embeddings, lengths = embedding_model(sentences)
            output = model(embeddings, lengths, sent_level_features=features)
            loss = criterion(output, labels)
            
            loss.backward()
            losses.append(loss.item())

            optimizer.step()
            optimizer.zero_grad()

        model.eval()
        y_pred, y_true = [], []
        with torch.no_grad():
            for val_ids, val_sentences, val_labels, val_features in val_loader:
                val_labels = val_labels.float().to(device)
                val_features = val_features.to(device)
                
                val_embeddings, val_lengths = embedding_model(val_sentences)
                pred = model(val_embeddings, val_lengths, sent_level_features=val_features)
                val_loss = criterion(pred, val_labels)
                val_losses.append(val_loss.item())
                
                pred = torch.sigmoid(pred)
                
                pred = (pred > threshold).int()
                y_pred.extend(pred.tolist())
                y_true.extend(val_labels.tolist())
        
#         print('epoch: ', epoch)
#         print('avg train loss: ', sum(losses) / len(losses))
#         print('avg val loss: ', sum(val_losses) / len(val_losses))
#         print(classification_report(y_true, y_pred, digits=6))
        cr = classification_report(y_true, y_pred, digits=6, output_dict=True, zero_division=0)
        
        val_loss = np.average(val_losses)
        early_stopping(val_loss, model, acomp_metrics={'recall_p': cr['1.0']['recall']})
        
        if early_stopping.early_stop:
            break

#         trial.report(recall_p, epoch)

#         # Handle pruning based on the intermediate value.
#         if trial.should_prune():
#             raise optuna.exceptions.TrialPruned()
    recall_p = early_stopping.acomp_metrics['recall_p'] if early_stopping.acomp_metrics else 0.0
    "Done."
    return recall_p

In [8]:
load_data()

In [ ]:
# optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))

# needed for GridSampler
# search_space = {
#     'batch_size': [16, 32, 64],
#     'pooling_strategy': ['last_four', 'last_four_sum', 'second_last'],
# #     'should_scale_emb': [False, True],
#     'dropout': [i/100 for i in range(0, 51, 5)],
#     'hidden_dim': [128, 256, 512],
#     'optimizer_weigth_decay': [i/10000 for i in range(11)],
#     'learning_rate': round_to_first_non_zero([i/100000 for i in range_inc(0, 100000, 1, 10)]),
#     'pos_weight': [1.0, train_uw_ratio]
# }
feature_search_space = {
    'stopwords_type': ['wstop', 'wostop'],
    'from_selection': [True, False],
    'pos_feature_type': ['sum', 'onehot', 'none'],
    'tag_feature_type': ['sum', 'onehot', 'none'],
    'word_count_feature_type': ['count_words', 'none'],
#     'word_level_feature_type': ['dep', 'triplet']
}
# print(search_space)
params = {
    'batch_size': 32,
    'pooling_strategy': 'second_last',
    'dropout': 0.39,
    'hidden_dim': 256,
    'w_seq': True,
    'lr': 0.004118121,
    'opt_weight_decay': 0.024460049,
    'pos_weight': train_uw_ratio,
}


study = optuna.create_study(
    study_name=f'bi-lstm_featOptim_wAtt_sTPE_pNone_df{dataset_frac}_wf{worthy_frac}',
#     sampler=optuna.samplers.TPESampler(),
    sampler=optuna.samplers.GridSampler(feature_search_space),
#     pruner=optuna.pruners.MedianPruner(),
    direction='maximize'
)
logf_path = p.join(LOG_DIR_PATH, f'{study.study_name}.log')
study.optimize(objective, n_trials=200)

study_path = os.path.join(optim_path, f'{study.study_name}.pkl')
torch.save(study, study_path)
torch.save(params, f'{os.path.join(optim_path, study.study_name)}_params.pkl')

In [10]:
loaded_study = torch.load(study_path)

print(loaded_study.best_trial.params)
print(loaded_study.best_trial)

{'stopwords_type': 'wstop', 'from_selection': True, 'pos_feature_type': 'none', 'tag_feature_type': 'sum', 'word_count_feature_type': 'none'}
FrozenTrial(number=39, values=[0.9207920792079208], datetime_start=datetime.datetime(2022, 3, 27, 17, 57, 28, 983824), datetime_complete=datetime.datetime(2022, 3, 27, 18, 1, 41, 808323), params={'stopwords_type': 'wstop', 'from_selection': True, 'pos_feature_type': 'none', 'tag_feature_type': 'sum', 'word_count_feature_type': 'none'}, distributions={'stopwords_type': CategoricalDistribution(choices=('wstop', 'wostop')), 'from_selection': CategoricalDistribution(choices=(True, False)), 'pos_feature_type': CategoricalDistribution(choices=('sum', 'onehot', 'none')), 'tag_feature_type': CategoricalDistribution(choices=('sum', 'onehot', 'none')), 'word_count_feature_type': CategoricalDistribution(choices=('count_words', 'none'))}, user_attrs={}, system_attrs={'search_space': OrderedDict([('from_selection', [False, True]), ('pos_feature_type', ['none'